In [1]:
# nuclio: ignore
import nuclio

In [2]:
%nuclio config kind = "job"
%nuclio config spec.image = "mlrun/ml-models"

%nuclio: setting kind to 'job'
%nuclio: setting spec.image to 'mlrun/ml-models'


In [3]:
# Copyright 2018 Iguazio
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import numpy as np
import pandas as pd
from mlrun.execution import MLClientCtx


def load_dataset(
        context: MLClientCtx,
        dataset: str,
        name: str = '',
        file_ext: str = 'parquet',
        params: dict = {}
) -> None:
    """Loads a scikit-learn toy dataset for classification or regression

    The following datasets are available ('name' : desription):

        'boston'          : boston house-prices dataset (regression)
        'iris'            : iris dataset (classification)
        'diabetes'        : diabetes dataset (regression)
        'digits'          : digits dataset (classification)
        'linnerud'        : linnerud dataset (multivariate regression)
        'wine'            : wine dataset (classification)
        'breast_cancer'   : breast cancer wisconsin dataset (classification)

    The scikit-learn functions return a data bunch including the following items:
    - data              the features matrix
    - target            the ground truth labels
    - DESCR             a description of the dataset
    - feature_names     header for data

    The features (and their names) are stored with the target labels in a DataFrame.

    For further details see https://scikit-learn.org/stable/datasets/index.html#toy-datasets

    :param context:    function execution context
    :param dataset:    name of the dataset to load
    :param name:       artifact name (defaults to dataset)
    :param file_ext:   output file_ext: parquet or csv
    :param params:     params of the sklearn load_data method
    """
    dataset = str(dataset)
    # reach into module and import the appropriate load_xxx function
    pkg_module = 'sklearn.datasets'
    fname = f'load_{dataset}'

    pkg_module = __import__(pkg_module, fromlist=[fname])
    load_data_fn = getattr(pkg_module, fname)

    data = load_data_fn(**params)
    feature_names = data['feature_names']

    # create the toy dataset
    xy = np.concatenate([data['data'], data['target'].reshape(-1, 1)], axis=1)
    if hasattr(feature_names, 'append'):
        # its a list
        feature_names.append('labels')
    else:
        # its an array
        feature_names = np.append(feature_names, 'labels')
    df = pd.DataFrame(data=xy, columns=feature_names)

    # log and upload the dataset
    context.log_dataset(name or dataset, df=df, format=file_ext, index=False)

In [4]:
# nuclio: end-code

### mlconfig

In [5]:
from mlrun import mlconf

mlconf.dbpath = mlconf.dbpath or './'

vcs_branch = 'development'
base_vcs = f'https://raw.githubusercontent.com/mlrun/functions/{vcs_branch}/'

mlconf.hub_url = mlconf.hub_url or base_vcs + f'{name}/function.yaml'

import os
mlconf.artifact_path = mlconf.artifact_path or f'{os.environ["V3IO_HOME"]}/artifacts'

### save

In [6]:
from mlrun import code_to_function 
# create job function object from notebook code
fn = code_to_function("load_dataset")

# add metadata (for templates and reuse)
fn.spec.default_handler = "load_dataset"
fn.spec.description = "load a toy dataset from scikit-learn"
fn.metadata.categories = ["datagen", "filesutils"]
fn.metadata.labels = {"author": "yjb"}

fn.save()
fn.export("function.yaml")

[mlrun] 2020-04-30 20:43:34,358 saving function: load-dataset, tag: latest
[mlrun] 2020-04-30 20:43:34,385 function spec saved to path: function.yaml


## tests

In [7]:
from mlrun import import_function

func = import_function("hub://load_dataset")

if "V3IO_HOME" in list(os.environ):
    from mlrun import mount_v3io
    func.apply(mount_v3io())
else:
    # is you set up mlrun using the instructions at https://github.com/mlrun/mlrun/blob/master/hack/local/README.md
    from mlrun.platforms import mount_pvc
    func.apply(mount_pvc('nfsvol', 'nfsvol', '/home/joyan/data'))

In [8]:
from mlrun import NewTask    

task_params = {
    "name"   : "tasks load toy dataset", 
    "params" : {"dataset"     : "wine"}}

### run remotely

In [9]:
run = func.run(NewTask(**task_params), artifact_path=mlconf.artifact_path)

[mlrun] 2020-04-30 20:43:34,438 starting run tasks load toy dataset uid=d1162e336d66482bbda7532038774e41  -> http://mlrun-api:8080
[mlrun] 2020-04-30 20:43:34,524 Job is running in the background, pod: tasks-load-toy-dataset-pkbpk
[mlrun] 2020-04-30 20:43:38,465 log artifact wine at /User/artifacts/wine.parquet, size: 17997, db: Y

[mlrun] 2020-04-30 20:43:38,504 run executed, status=completed
final state: succeeded


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...38774e41,0,Apr 30 20:43:38,completed,tasks load toy dataset,host=tasks-load-toy-dataset-pkbpkkind=jobowner=adminv3io_user=admin,,dataset=wine,,wine


to track results use .show() or .logs() or in CLI: 
!mlrun get run d1162e336d66482bbda7532038774e41  , !mlrun logs d1162e336d66482bbda7532038774e41 
[mlrun] 2020-04-30 20:43:40,752 run executed, status=completed


### or locally

In [10]:
from mlrun import run_local

In [11]:
for dataset in ["wine", "iris", "breast_cancer"]:
    run_local(handler=load_dataset,
              inputs={"dataset": dataset}, artifact_path=mlconf.artifact_path)

[mlrun] 2020-04-30 20:43:40,777 starting run mlrun-30b88e-load_dataset uid=a1ced97026824d32bfc02af895d11c44  -> http://mlrun-api:8080
[mlrun] 2020-04-30 20:43:41,331 log artifact wine at /User/artifacts/wine.parquet, size: 18006, db: Y



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...95d11c44,0,Apr 30 20:43:40,completed,mlrun-30b88e-load_dataset,v3io_user=adminkind=handlerowner=adminhost=jupyter-6c5fccf844-gxlrw,dataset,,,wine


to track results use .show() or .logs() or in CLI: 
!mlrun get run a1ced97026824d32bfc02af895d11c44 --project default , !mlrun logs a1ced97026824d32bfc02af895d11c44 --project default
[mlrun] 2020-04-30 20:43:41,395 run executed, status=completed
[mlrun] 2020-04-30 20:43:41,404 starting run mlrun-e7367c-load_dataset uid=131406535a4946bfb4f7edcdca77d75e  -> http://mlrun-api:8080
[mlrun] 2020-04-30 20:43:41,495 log artifact iris at /User/artifacts/iris.parquet, size: 5049, db: Y



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...ca77d75e,0,Apr 30 20:43:41,completed,mlrun-e7367c-load_dataset,v3io_user=adminkind=handlerowner=adminhost=jupyter-6c5fccf844-gxlrw,dataset,,,iris


to track results use .show() or .logs() or in CLI: 
!mlrun get run 131406535a4946bfb4f7edcdca77d75e --project default , !mlrun logs 131406535a4946bfb4f7edcdca77d75e --project default
[mlrun] 2020-04-30 20:43:41,540 run executed, status=completed
[mlrun] 2020-04-30 20:43:41,549 starting run mlrun-fcee35-load_dataset uid=2ecb3916190a4493aad2d75265df98d0  -> http://mlrun-api:8080
[mlrun] 2020-04-30 20:43:41,869 log artifact breast_cancer at /User/artifacts/breast_cancer.parquet, size: 146754, db: Y



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...65df98d0,0,Apr 30 20:43:41,completed,mlrun-fcee35-load_dataset,v3io_user=adminkind=handlerowner=adminhost=jupyter-6c5fccf844-gxlrw,dataset,,,breast_cancer


to track results use .show() or .logs() or in CLI: 
!mlrun get run 2ecb3916190a4493aad2d75265df98d0 --project default , !mlrun logs 2ecb3916190a4493aad2d75265df98d0 --project default
[mlrun] 2020-04-30 20:43:41,950 run executed, status=completed
